In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [89]:
Bundes_liga_19_20 = pd.read_csv("data_auto/Bundes_lige2019-2020.csv")
Bundes_liga_20_21 = pd.read_csv("data_auto/Bundes_lige2020-2021.csv")
Bundes_liga_21_22 = pd.read_csv("data_auto/Bundes_lige2021-2022.csv")
Bundes_liga_22_23 = pd.read_csv("data_auto/Bundes_lige2022-2023.csv")

Premier_liga19_20 = pd.read_csv("data_auto/Premier_liga2019-2020.csv")
Premier_liga20_21 = pd.read_csv("data_auto/Premier_liga2020-2021.csv")
Premier_liga21_22 = pd.read_csv("data_auto/Premier_liga2021-2022.csv")
Premier_liga22_23 = pd.read_csv("data_auto/Premier_liga2022-2023.csv")

Seria_A_liga19_20 = pd.read_csv("data_auto/Seria_A_liga2019-2020.csv")
Seria_A_liga20_21 = pd.read_csv("data_auto/Seria_A_liga2020-2021.csv")
Seria_A_liga21_22 = pd.read_csv("data_auto/Seria_A_liga2021-2022.csv")
Seria_A_liga22_23 = pd.read_csv("data_auto/Seria_A_liga2022-2023.csv")


In [90]:
display(Bundes_liga_19_20.shape)
display(Bundes_liga_20_21.shape)
display(Bundes_liga_21_22.shape)
display(Bundes_liga_22_23.shape)
n = [Bundes_liga_19_20, Bundes_liga_20_21, Bundes_liga_21_22, Bundes_liga_22_23]

(308, 74)

(308, 73)

(308, 73)

(308, 73)

In [91]:
def point_vizual(point):#на вход принимает словрь с команда:очки
    point_df = pd.DataFrame({
        'счет': list(point.values())
    })
    point_df.index = list(d.keys())
    point_df.sort_values(by="счет",axis=0,ascending=False)
    return(point_df)

def point_bund(df):
    n = list(df["Команда1"].value_counts().index)
    d = dict()
    for i in n:
        d.update({i:0})
    df = df.sort_values(by="Tour",axis=0)
    for i in list(df["id"]):
        num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
        
        df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
        df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
        
        if num[1] > num[3]:
            d[num[0]] +=  3
        elif num[1] < num[3]:
            d[num[2]] +=  3
        else:
            d[num[0]] +=  1
            d[num[2]] +=  1
    return df

def preobr(df):
    if 'Unnamed: 0.1' in list(df.columns):
        df = df.drop('Unnamed: 0.1',axis=1)
    if 'Unnamed: 0' in list(df.columns):
        df = df.drop('Unnamed: 0',axis=1)
    
    df = df.drop('Матчи 1',axis=1)
    df = df.drop('Матчи 2',axis=1)
    df = df.astype({'счет1': 'int32'})
    df = df.astype({'счет2': 'int32'})

    df["тотал"] = df["счет1"] + df["счет2"]
    df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
    df['Температура'] = df['Температура'].apply(lambda x: int(x[1:-2]))
    
    df.loc[df["id"] == 14898101 ,['Офсайды  (соперник) 2']] = 4.00
    df.loc[df["id"] == 14898101 ,['Офсайды 2']] = 1.00
    
    #mask_1 = df["id"] == 14898101
    #df['Офсайды  (соперник) 2'][mask_1] = 4.00
    #df['Офсайды 2'][mask_1] = 1.00
        
    df["Point_1"] = 0
    df["Point_2"] = 0
    df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
    df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
    
    df = df[df["Tour"] != " стыковые матчи"]
    df = df[df["Tour"] != " понижение/повышение - финал"]
    df = df[df["Tour"] != " дополнительный матч"]
    df["Tour"] = df["Tour"].apply(lambda x: int(x.strip()[:-6]))
    
    df = df.astype({'Офсайды  (соперник) 2': 'float32'})
    df = df.astype({'Офсайды 2': 'float32'})

    df["Сухие матчи 1"] = df["Сухие матчи 1"].apply(lambda x: int(x.split()[-1]))
    df["Сухие матчи 2"] = df["Сухие матчи 2"].apply(lambda x: int(x.split()[0]))

    df["обе забьют 1"] = df["обе забьют 1"].apply(lambda x: int(x.split()[-1]))
    df["обе забьют 2"] = df["обе забьют 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал больше 2.5 1"] = df["Тотал больше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал больше 2.5 2"] = df["Тотал больше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал меньше 2.5 1"] = df["Тотал меньше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал меньше 2.5 2"] = df["Тотал меньше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Поражения 1"] = df["Поражения 1"].apply(lambda x: int(x.split()[-1]))
    df["Поражения 2"] = df["Поражения 2"].apply(lambda x: int(x.split()[0]))

    df["Ничьи 1"] = df["Ничьи 1"].apply(lambda x: int(x.split()[-1]))
    df["Ничьи 2"] = df["Ничьи 2"].apply(lambda x: int(x.split()[0]))

    df["Победы 1"] = df["Победы 1"].apply(lambda x: int(x.split()[-1]))
    df["Победы 2"] = df["Победы 2"].apply(lambda x: int(x.split()[0]))

    df["Отдых: дни (часы) 1"] = df["Отдых: дни (часы) 1"].apply(lambda x: float(x.split()[-1]))
    df["Отдых: дни (часы) 2"] = df["Отдых: дни (часы) 2"].apply(lambda x: float(x.split()[0]))
    
    df = point_bund(df)
    return df

In [92]:
Bundes_liga_19_20_cl = preobr(Bundes_liga_19_20)
Bundes_liga_20_21_cl = preobr(Bundes_liga_20_21)
Bundes_liga_21_22_cl = preobr(Bundes_liga_21_22)
Bundes_liga_22_23_cl = preobr(Bundes_liga_22_23)

Premier_liga19_20_cl = preobr(Premier_liga19_20)
Premier_liga20_21_cl = preobr(Premier_liga20_21)
Premier_liga21_22_cl = preobr(Premier_liga21_22)
Premier_liga22_23_cl = preobr(Premier_liga22_23)

Seria_A_liga19_20_cl = preobr(Seria_A_liga19_20)
Seria_A_liga20_21_cl = preobr(Seria_A_liga20_21)
Seria_A_liga21_22_cl = preobr(Seria_A_liga21_22)
Seria_A_liga22_23_cl = preobr(Seria_A_liga22_23)

In [93]:
display(Premier_liga19_20_cl[Premier_liga19_20_cl["Tour"] == 1][["Команда1", "счет1", "Команда2", "счет2"]])
display(Premier_liga19_20_cl[Premier_liga19_20_cl["Tour"] == 2][["Команда1", "Point_1", "Команда2", "Point_2"]])

,Команда1,счет1,Команда2,счет2
379,Ливерпуль,4,Норвич Сити,1
370,Ман Юнайтед,4,Челси,0
371,Лестер Сити,0,Вулверхэмптон,0
372,Ньюкасл,0,Арсенал,1
373,Тоттенхэм,3,Астон Вилла,1
378,Вест Хэм,0,Ман Сити,5
375,Бернли,3,Саутгемптон,0
376,Кристал Пэлас,0,Эвертон,0
377,Уотфорд,0,Брайтон,3
374,Борнмут,1,Шеффилд Ютд,1


,Команда1,Point_1,Команда2,Point_2
360,Вулверхэмптон,1,Ман Юнайтед,3
361,Челси,0,Лестер Сити,1
362,Шеффилд Ютд,1,Кристал Пэлас,1
363,Ман Сити,3,Тоттенхэм,3
364,Астон Вилла,0,Борнмут,1
365,Брайтон,3,Вест Хэм,0
366,Эвертон,1,Уотфорд,0
367,Норвич Сити,0,Ньюкасл,0
368,Саутгемптон,0,Ливерпуль,3
369,Арсенал,3,Бернли,3
